In [15]:
import re
import os
import numpy as np
import pandas as pd
from transformers import AutoTokenizer

In [16]:
rootPath = "/"
relTrainCSVpath = "CONDA/data/CONDA_train.csv"
relTestCSVpath = "CONDA/data/CONDA_test.csv"
relValidationCSVpath = "CONDA/data/CONDA_valid.csv"

In [17]:
trainDF = pd.read_csv(relTrainCSVpath)
testDF = pd.read_csv(relTestCSVpath)
valDF = pd.read_csv(relValidationCSVpath)

In [18]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [19]:
texts = trainDF['utterance']
labels = trainDF['intentClass']
d = {'texts':texts, 'labels': labels}
train_DF = pd.DataFrame(d)
train_DF = train_DF.fillna('O')
texts = train_DF["texts"].tolist()

In [20]:
# trainDF2 = trainDF['utterance'].map(lambda x: tokenizer(str(x), truncation=True))

#trainDF['utterance'] = trainDF['utterance'].map(lambda x: tokenizer(str(x), truncation=True))
#train_Str = trainDF.utterance.values.tolist()
train_encodings = tokenizer(texts, truncation=True, padding=True)

In [21]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [22]:
import evaluate

accuracy = evaluate.load("accuracy")

In [23]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [24]:
# print(trainDF['intentClass'][:20])
# from sklearn.preprocessing import LabelEncoder

# le = LabelEncoder()
# trainDF['intentClassLabel'] = le.fit_transform(trainDF['intentClass'])
# trainDF[:10]
train_DF["labels"].tail()

26916    O
26917    O
26918    O
26919    O
26920    O
Name: labels, dtype: object

In [25]:
id2label = {0: "A", 1: "E", 2: "I", 3: "O"}
label2id = {"A": 0, "E": 1, "I": 2, "O": 3}

In [26]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=4, id2label=id2label, label2id=label2id
)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.bias', 'classifier.w

In [27]:
texts = valDF['utterance']
labels = valDF['intentClass']
d = {'texts':texts, 'labels': labels}
test_DF = pd.DataFrame(d)
test_DF = test_DF.fillna('O')
val_texts = test_DF['texts'].tolist()

In [28]:
test_encodings = tokenizer(val_texts, truncation=True, padding=True) 

In [29]:
train_DF["labels"].replace('A',0,inplace=True)
train_DF["labels"].replace('E',1,inplace=True)
train_DF["labels"].replace('I',2,inplace=True)
train_DF["labels"].replace('O',3,inplace=True)
test_DF["labels"].replace('A',0,inplace=True)
test_DF["labels"].replace('E',1,inplace=True)
test_DF["labels"].replace('I',2,inplace=True)
test_DF["labels"].replace('O',3,inplace=True)


In [30]:
from torch.utils.data import Dataset
import torch


class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# class CustomDataset(Dataset):

#     def __init__(self, dataframe, tokenizer, max_len):
#         self.tokenizer = tokenizer
#         self.data = dataframe
#         # texts = dataframe.texts.tolist()
#         # self.texts = tokenizer(texts,truncation=True, padding=True)
#         self.texts = dataframe.texts
#         self.targets = self.data.labels
#         self.max_len = max_len

#     def __len__(self):
#         return len(self.texts)

#     def __getitem__(self, index):
#         # texts = str(self.texts[index])
#         # texts = " ".join(texts.split())

#         # # inputs = self.tokenizer.encode_plus(
#         # #     texts,
#         # #     None,
#         # #     add_special_tokens=True,
#         # #     max_length=self.max_len,
#         # #     pad_to_max_length=True,
#         # #     return_token_type_ids=True
#         # # )
#         # # ids = inputs['input_ids']
#         # mask = inputs['attention_mask']
#         # token_type_ids = inputs["token_type_ids"]


#         # return {
#         #     'ids': torch.tensor(ids, dtype=torch.long),
#         #     'mask': torch.tensor(mask, dtype=torch.long),
#         #     'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
#         #     'targets': torch.tensor(self.targets[index], dtype=torch.float)
#         #     }
#         item = {key: torch.tensor(val[index]) for key, val in self.texts.items()}
#         item['labels'] = torch.tensor(self.labels[index])
#         return item

In [31]:
trainDataset = CustomDataset(train_encodings, train_DF['labels'])
testDataset = CustomDataset(test_encodings,test_DF['labels'])

In [32]:
train_DF['labels'][7720]

3

In [33]:
training_args = TrainingArguments(
    output_dir="temp_model",
    learning_rate=2e-5,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=trainDataset,
    eval_dataset=testDataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

c:\Users\Karan\Documents\AI_COURSE\aasd4011-2023-spring\venv\pytorch\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
  0%|          | 0/842 [00:00<?, ?it/s]You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
                                                 
 50%|█████     | 421/842 [05:15<04:34,  1.54it/s]

{'eval_loss': 0.2785699963569641, 'eval_accuracy': 0.9214397147314464, 'eval_runtime': 35.7399, 'eval_samples_per_second': 251.092, 'eval_steps_per_second': 3.945, 'epoch': 1.0}


 59%|█████▉    | 500/842 [06:16<04:18,  1.33it/s]  

{'loss': 0.3726, 'learning_rate': 8.12351543942993e-06, 'epoch': 1.19}


                                                 
100%|██████████| 842/842 [11:18<00:00,  1.41it/s]

{'eval_loss': 0.26785334944725037, 'eval_accuracy': 0.9231112101626923, 'eval_runtime': 38.9431, 'eval_samples_per_second': 230.439, 'eval_steps_per_second': 3.621, 'epoch': 2.0}


100%|██████████| 842/842 [11:19<00:00,  1.24it/s]

{'train_runtime': 679.927, 'train_samples_per_second': 79.188, 'train_steps_per_second': 1.238, 'train_loss': 0.31951415000788674, 'epoch': 2.0}


TrainOutput(global_step=842, training_loss=0.31951415000788674, metrics={'train_runtime': 679.927, 'train_samples_per_second': 79.188, 'train_steps_per_second': 1.238, 'train_loss': 0.31951415000788674, 'epoch': 2.0})